Este exemplo usa o pacote Surprise para construir um modelo baseado em modelo usando o algoritmo SVD (Singular Value Decomposition) e o pacote sklearn para construir um modelo baseado em memória usando o algoritmo NearestNeighbors. Ele então combina as recomendações dos dois modelos para fornecer uma lista final de recomendações ao usuário.

In [ ]:
!pip install surprise -q

In [ ]:
import pandas as pd
from surprise import SVD
from sklearn.neighbors import NearestNeighbors
from surprise.model_selection import train_test_split

In [ ]:
from surprise import Reader, Dataset, KNNWithMeans

In [ ]:
# Carregando dados de avaliações de filmes
ratings = pd.read_csv("/content/drive/MyDrive/recomendacao/ratings_small.csv")

In [ ]:
#data = Dataset.load_from_file('ratings.csv', reader=Reader(line_format='user item rating', sep=','))

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

Neste exemplo, primeiro os dados de avaliações são carregados do arquivo CSV e são configurados para o objeto Reader. Em seguida, os dados de avaliações são carregados para o objeto Dataset para serem utilizados pelos algoritmos de CF. Depois, os dados são divididos em conjunto de treinamento e teste para avaliar o desempenho do modelo. O modelo baseado em memória (KNN) é treinado usando o método fit e suas previsões são obtidas com o método test. Finalmente, o desempenho do modelo é avaliado usando o erro quadrático médio (MSE). O mesmo

In [ ]:
# Treine o modelo de CF baseado em memória (KNN)
algo_knn = KNNWithMeans(k=30, sim_options={'name': 'pearson', 'user_based': False})
algo_knn.fit(trainset)

# Obtenha as previsões para o conjunto de teste
test_pred = algo_knn.test(testset)

# Avalie o desempenho do modelo com o erro quadrático médio (MSE)
from surprise import accuracy
acc = accuracy.rmse(test_pred)
print("O erro quadrático médio do modelo baseado em memória é:", acc)

# Treine o modelo de CF baseado em modelo (SVD)
from surprise import SVD
algo_svd = SVD()
algo_svd.fit(trainset)

# Obtenha as previsões para o conjunto de teste
test_pred = algo_svd.test(testset)

# Avalie o desempenho do modelo com o erro quadrático médio (MSE)
acc = accuracy.rmse(test_pred)
print("O erro quadrático médio do modelo baseado em modelo é:", acc)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9064
O erro quadrático médio do modelo baseado em memória é: 0.9064406741610493
RMSE: 0.8655
O erro quadrático médio do modelo baseado em modelo é: 0.8655297078450244


In [ ]:
# Obter previsões para um usuário dado
user_id = 1
movie_id = 6
knn_predictions = algo_knn.predict(user_id, 20)
svd_predictions = algo_svd.predict(user_id, 20)

In [ ]:
svd_predictions.est

3.661834629137796

In [ ]:
knn_predictions.est

3.2214834517835094

In [ ]:
user_1 = ratings.query("userId == 1")

In [ ]:
user_1.query("movieId == 6")

,userId,movieId,rating,timestamp
2,1,6,4.0,964982224


In [ ]:
# Combinar as previsões usando média ponderada
weighted_predictions = (knn_predictions.est * 0.7) + (svd_predictions.est * 0.3)

In [ ]:
weighted_predictions

3.353588804989795

## Para Vários Filmes

In [ ]:
# Obtendo previsões para um usuário dado
user_id = 1
movie_ids = [1, 2, 47, 4, 5]
predictions_knn = [algo_knn.predict(user_id, movie_id, verbose=False).est for movie_id in movie_ids]
predictions_svd = [algo_svd.predict(user_id, movie_id, verbose=False).est for movie_id in movie_ids]

In [ ]:
predictions_knn

[4.7042166497262965,
 4.317587480648559,
 4.803595113315072,
 2.8627918152601604,
 4.218906910155301]

In [ ]:
predictions_svd

[4.600946839409213,
 4.20854812050185,
 4.603107240536228,
 4.079283416926583,
 3.8422926572228455]

In [ ]:
weighted_predictions = [(knn_predictions * 0.7) + (svd_predictions * 0.3) for knn_predictions,svd_predictions in zip(predictions_knn, predictions_svd)]

In [ ]:
weighted_predictions

[4.673235706631171,
 4.284875672604546,
 4.743448751481418,
 3.2277392957600872,
 4.105922634275564]

In [ ]:
my_recs = []
for iid in movie_ids:
  predictions_knn = algo_knn.predict(user_id, iid, verbose=False).est 
  predictions_svd = algo_svd.predict(user_id, iid, verbose=False).est 
  weighted_predictions = (knn_predictions.est * 0.7) + (svd_predictions.est * 0.3) 
  my_recs.append((iid, weighted_predictions, predictions_svd, predictions_knn))

pd.DataFrame(my_recs, columns=['iid', 'weighted predictions', "svd", "knn"]).sort_values('weighted predictions', ascending=False).head(10)

,iid,weighted predictions,svd,knn
0,1,3.353589,4.600947,4.704217
1,2,3.353589,4.208548,4.317587
2,47,3.353589,4.603107,4.803595
3,4,3.353589,4.079283,2.862792
4,5,3.353589,3.842293,4.218907


## Usando Robust SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

# Carregue sua matriz de avaliações de filmes
ratings_matrix = ...

# Decomponha a matriz usando o SVD
svd = TruncatedSVD(n_components=100)
U = svd.fit_transform(ratings_matrix)

# Agrupe os usuários usando o k-means
kmeans = KMeans(n_clusters=5)
user_groups = kmeans.fit_predict(U)

# Faça recomendações personalizadas para cada grupo de usuários
for group in range(5):
    group_users = np.where(user_groups == group)
    group_movies = ratings_matrix[group_users].mean(axis=0) 
    recommended_movies = np.argsort(-group_movies)[:10]
    print("Recomendações para o grupo {}: {}".format(group, recommended_movies))